# Código desenvolvido em Java para criação da aplicação para contar palavras. Projeto inicial chamado ContaPalavras contendo a estrutura-base das classes

### 1-Desenvolvimento das 3 classes Mapper, Reducer e Driver para implementação da aplicação

In [ ]:
// Desenvolvimento da classe Driver ContaPalavrasDriver.java

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;


public class ContaPalavrasDriver {

	
	
	public static void main(String[] args) throws Exception {
		
		Configuration conf = new Configuration();
		// o nome "contapalavrasdriver" serve para quando verificar o log identificar a qual job se refere
		Job job = Job.getInstance(conf, "contapalavrasdriver");
		
		job.setJarByClass(ContaPalavrasDriver.class);
		job.setMapperClass(ContaPalavrasMap.class);
		job.setReducerClass(ContaPalavrasReduce.class);
		job.setOutputKeyClass(Text.class);// titpo de dado da chave
		job.setOutputValueClass(IntWritable.class);
		//job.setMapOutputKeyClass(theClass); indica tipo de dado do map
		//job.setInputFormatClass(cls); estamos usando a default, que é texto
		FileInputFormat.addInputPath(job, new Path(args[0]));
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		System.exit(job.waitForCompletion(true) ? 0 : 1);
		
			    
	  } 
}

In [ ]:
// Desenvolvimento da classe Mapper ContaPalavrasMap.java

import java.io.IOException;
import java.util.StringTokenizer;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;

                                              // tipo da chave e valor de entrada, tipo da chave e valor da saida
public class ContaPalavrasMap extends Mapper<Object, Text, Text, IntWritable>{
	// final indica que não vou alterar a variavel
	private final static IntWritable numeroum = new IntWritable(1);
	// context serve para escrever os metodos intermediarios
    public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
    	StringTokenizer itr = new StringTokenizer(value.toString().replaceAll("[^a-zA-Z ]", "").toLowerCase());
    	while (itr.hasMoreTokens()) {
    	context.write(new Text(itr.nextToken()), numeroum);
    	}
    }   
 }

In [ ]:
// Desesenvolvimento da classe Reducer ContaPalavrasReduce.java

import java.io.IOException;
import java.util.StringTokenizer;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;


public class ContaPalavrasReduce extends Reducer<Text,IntWritable,Text,IntWritable> {

	public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
		int soma = 0;
		for (IntWritable val : values) {
		soma += val.get();
		}
		context.write(key, new IntWritable(soma));
	}
}

### 2-Geração do arquivo JAR da aplicação e definição do path:

Exemplo: /home/cloudera/Documents/contapalavras.jar

### 3-Execução do job JAR no ambiente Hadoop e consumo do arquivo bases/base_tw.txt já armazenado no HDFS 

 hadoop jar ~/Documents/contapalavras.jar ContaPalavrasDriver bases/base_tw.txt twitter/saida

### 4-Visualização do resultado da aplicação

hdfs dfs -text twitter/saida/part-r-00000